# Connect to Asana instance

In [ ]:
import os
import asana
from pathlib import Path
import math
import datetime
import pandas as pd

# @icaoberg Read secrets. Assumes file exists in ~/.ASANA_SECRETS
file = str(Path.home()) + os.sep + '.ASANA_SECRETS'
exec(open(file).read())

client = asana.Client.access_token(PERSONAL_ACCESS_TOKEN)

## Information and parameters

In [ ]:
#SOFTWARE THAT I WANT TO INSTALL
username='1183587634011025' #icaoberg
workspace='1132331877475002' #PSC
team='1192784280891412' #Brain Image Library

maindelta=7
when=datetime.date.today() + datetime.timedelta(days=maindelta+1)

## Public datasets information

In [ ]:
file = 'summary_metadata.tsv'
df = pd.read_csv(file, sep='\t')
number_of_rows = len(df)
partitions = math.ceil(number_of_rows/500)

# Create project

In [ ]:
params = {'name':'Build inventories for all public datasets', 
          'color':'dark-green', \
          'archived':False, \
          'is_template':False, \
          'public': True, \
          'notes':'Download latest summary metadata TSV file and build inventories for all public datasets.', \
          'default_view':'calendar', \
          'due_on':str(when + datetime.timedelta(days=7)), \
          'start_on':str(when), \
          'created_by':client.users.get_user(username)['name']}

answer = client.projects.create_in_team( team, params )
project = answer['gid']

# Create tasks

In [ ]:
strings = []
deltas = []

strings.append(f'Divide all public datasets into {str(partitions)}')
deltas.extend([1])

for i in range(partitions):
    strings.append(f'Build inventories on partition {str(i+1)}')
    deltas.extend([7])

for delta, string in zip(deltas, strings):
    params={'name': string, \
            'assignee': username, \
            'projects': [project], \
            'due_on': str(when+datetime.timedelta(days=maindelta+1+delta)), \
            'start_on': str(when)}

    answer = client.tasks.create_in_workspace(workspace,params)

# Rename default section

In [ ]:
answer = client.sections.find_by_project(project)
section = next(answer)
sectionid = section['gid']
params = {'name':'To Do'}

answer = client.sections.update( sectionid, params )

# Create sections

In [ ]:
strings = { 'In Progress', \
           'Done'}

for string in strings:
    params={'name': string}
    answer = client.sections.create_in_project(project, params)